In [2]:
import arcpy;
import os,sys;
import requests;

start_date  = datetime.date(1990,1,1);
end_date    = datetime.date(2020,12,31);

results_fgdb = os.getcwd() + os.sep + '..'+ os.sep + 'results.gdb';
target_dir   = os.getcwd() + os.sep + 'nldas';

if not os.path.exists(target_dir):
    os.mkdir(target_dir);

stations = results_fgdb  + os.sep + 'SWC_Station_Universe';
stations_cnt = arcpy.GetCount_management(stations)[0];
print("  Initial SWC Station Universe Count: " + str(stations_cnt));


  Initial SWC Station Universe Count: 5144


In [3]:
class NLDAS:
    DEGREES_PER_GRID_CELL = 1.0 / 8.0
    WESTMOST_GRID_EDGE = -125.0
    SOUTHMOST_GRID_EDGE = 25.0
    WESTMOST_GRID_CENTER = WESTMOST_GRID_EDGE + DEGREES_PER_GRID_CELL / 2.0
    SOUTHMOST_GRID_CENTER = SOUTHMOST_GRID_EDGE + DEGREES_PER_GRID_CELL / 2.0

    def grid_cell_from_lat_lon(lat, lon):
        """
        Takes latitude and longitude in degrees and returns
        string of x/y NLDAS grid cell
        """
        x = int(round((lon - NLDAS.WESTMOST_GRID_CENTER)
                    / NLDAS.DEGREES_PER_GRID_CELL))
        y = int(round((lat - NLDAS.SOUTHMOST_GRID_CENTER)
                    / NLDAS.DEGREES_PER_GRID_CELL))
        return str(x), str(y);


In [4]:
%%time

raw_files_dir = os.getcwd() + os.sep + 'nldas' + os.sep + 'raw';

if not os.path.exists(raw_files_dir):
    os.mkdir(raw_files_dir);

with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId','SHAPE@']
) as incur:

    for row in incur:
        
        point = row[1].firstPoint;
        lat  = round(point.Y,8);
        lon  = round(point.X,8);
        name = row[0];
        
        target_file = raw_files_dir + os.sep + name + '_nldas.txt';
        
        if 25 < lat < 53 and -125 < lon < -63 and not os.path.exists(target_file):
            
            x_grid,y_grid = NLDAS.grid_cell_from_lat_lon(lat,lon);
            
            url = "https://hydro1.sci.gsfc.nasa.gov/daac-bin/access/"        \
                + "timeseries.cgi?variable=NLDAS:NLDAS_FORA0125_H.002:TMP2m" \
                + "&startDate=" + start_date.strftime("%Y-%m-%dT00")         \
                + "&endDate="   + end_date.strftime("%Y-%m-%dT00")           \
                + "&location=NLDAS:X" + x_grid + "-Y" + y_grid + "&type=asc2";
            
            with open(target_file,'wb') as f:

                with requests.get(
                     url
                    ,stream = True
                ) as r:
                    for line in r.iter_lines():
                        f.write(line+'\n'.encode());
                            
            

Wall time: 4h 15min 15s


In [27]:
target_file = target_dir + os.sep + 'nldas_20210818.csv';

if os.path.exists(target_file):
    os.remove(target_file);

with arcpy.da.SearchCursor(
     in_table     = results_fgdb + os.sep + 'SWC_Station_Universe'
    ,field_names  = ['StationId']
) as incur:
    
    with open(target_file,'w') as outcur:
    
        for row in incur:
            name = row[0];
            source_file = raw_files_dir + os.sep + name + '_nldas.txt';

            if os.path.exists(source_file) and os.path.getsize(source_file) > 300:

                with open(source_file,'r') as f:

                    for line in f:

                        if line[0:7] in ['      1','      2']:

                            dt  = datetime.datetime.strptime(line[6:20],"%Y-%m-%d %HZ");
                            tpk = float(line[24:50]) ;

                            if tpk > -9998:
                                tpf = (tpk - 273.15) * 1.8000 + 32.00;

                                outcur.write('"' + name + '","' + str(dt) + '",' + str(round(tpf,8)) + '\n');
    